### Solutions

#### Question 1

Write a custom class that will be used to model a single bank account.

Your class should implement functionality to:
- allow initialization with values for `first_name`, `last_name`, `account_number`, `balance`, `is_overdraft_allowed`
- keep track of a "ledger" that keeps a record all transactions (just use a list to keep track of this)
    - at a minimum it should keep track of the transaction date (the current UTC datetime) and the amount (positive, or negative to indicate deposits/withdrawals) - later you could add tracking the running balance as well.
- provide read-only properties for `first_name`, `last_name`, `account_number` and `balance`
- provide a property to access the ledger in such a way that a user of this class cannot mutate the ledger directly
- provide a read-write property for `is_overdraft_allowed` that indicates whether overdrafts are allowed on the account.
- provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that:
    - verify withdrawals against available balance and `is_overdraft_allowed` flag
        - if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception.
        - if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance).
    - add an entry to the ledger with a current UTC timestamp (positive or negative to indicate credit/debit)
    - keeps the available balance updated
- implements a good string representation for the instance (maybe something like `first_name last_name (account_number): balance`

Feel free to expand on the minimum definition I have given here and enhance your custom class.

### Solution

In [1]:
from datetime import datetime, timezone

class OverdraftNotAllowed(Exception):
    pass

class BankAccount:
    def __init__(self, first_name, last_name, account_number, balance, is_overdraft_allowed):
        self._first_name = first_name
        self._last_name = last_name
        self._account_number = account_number
        self._balance = balance
        self._is_overdraft_allowed = is_overdraft_allowed
        self._ledger = []

    @property
    def first_name(self):
        return self._first_name

    @property
    def last_name(self):
        return self._last_name

    @property
    def account_number(self):
        return self._account_number

    @property
    def balance(self):
        return self._balance

    @property
    def ledger(self):
        return self._ledger.copy()

    @property
    def is_overdraft_allowed(self):
        return self._is_overdraft_allowed

    @is_overdraft_allowed.setter
    def is_overdraft_allowed(self, value):
        self._is_overdraft_allowed = value

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive.")
        
        if not self._is_overdraft_allowed and self._balance < amount:
            raise OverdraftNotAllowed("Overdrafts are not allowed.")

        self._balance -= amount
        self._ledger.append({"date": datetime.now(timezone.utc), "amount": -amount})
        
    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive.")
        
        self._balance += amount
        self._ledger.append({"date": datetime.now(timezone.utc), "amount": amount})

    def __str__(self):
        return f"{self._first_name} {self._last_name} ({self._account_number}): {self._balance}"



In [3]:
# Creating an instance of BankAccount
account = BankAccount("Shrawan", "Kumawat", "123456789", 1000, True)

# Accessing properties
print(account.first_name)  
print(account.last_name)  
print(account.account_number)  
print(account.balance)     
print(account.is_overdraft_allowed)  

# Depositing and Withdrawing
account.deposit(500)
account.withdraw(200)
print(account.balance)     

# Accessing the ledger
ledger = account.ledger
for transaction in ledger:
    print(transaction)

# Modifying overdraft allowance
account.is_overdraft_allowed = False

# Attempting to withdraw more than the available balance (without overdrafts)
try:
    account.withdraw(2000)
except OverdraftNotAllowed as e:
    print(e)  # Overdrafts are not allowed.

# Attempting to withdraw a negative amount
try:
    account.withdraw(-500)
except ValueError as e:
    print(e)  # Amount must be positive.

# String representation of the account
print(str(account))  

Shrawan
Kumawat
123456789
1000
True
1300
{'date': datetime.datetime(2023, 5, 22, 11, 29, 15, 515263, tzinfo=datetime.timezone.utc), 'amount': 500}
{'date': datetime.datetime(2023, 5, 22, 11, 29, 15, 515263, tzinfo=datetime.timezone.utc), 'amount': -200}
Overdrafts are not allowed.
Amount must be positive.
Shrawan Kumawat (123456789): 1300


#### Question 2

Expand on your class above to implement equality (`==`) comparisons between instances of your class.

Two accounts should be considered equal if the account numbers are the same.

### Solution

In [4]:
from datetime import datetime, timezone

class OverdraftNotAllowed(Exception):
    pass

class BankAccount:
    def __init__(self, first_name, last_name, account_number, balance, is_overdraft_allowed):
        self._first_name = first_name
        self._last_name = last_name
        self._account_number = account_number
        self._balance = balance
        self._is_overdraft_allowed = is_overdraft_allowed
        self._ledger = []

    @property
    def first_name(self):
        return self._first_name

    @property
    def last_name(self):
        return self._last_name

    @property
    def account_number(self):
        return self._account_number

    @property
    def balance(self):
        return self._balance

    @property
    def ledger(self):
        return self._ledger.copy()

    @property
    def is_overdraft_allowed(self):
        return self._is_overdraft_allowed

    @is_overdraft_allowed.setter
    def is_overdraft_allowed(self, value):
        self._is_overdraft_allowed = value

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive.")

        if not self._is_overdraft_allowed and self._balance < amount:
            raise OverdraftNotAllowed("Overdrafts are not allowed.")

        self._balance -= amount
        self._ledger.append({"date": datetime.now(timezone.utc), "amount": -amount})

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive.")

        self._balance += amount
        self._ledger.append({"date": datetime.now(timezone.utc), "amount": amount})

    def __eq__(self, other):
        if isinstance(other, BankAccount):
            return self._account_number == other._account_number
        return False

    def __str__(self):
        return f"{self._first_name} {self._last_name} ({self._account_number}): {self._balance}"


In [5]:
# Creating 3 BankAccount instances
account1 = BankAccount("Shrawan", "Kumawat", "123456789", 1000, True)
account2 = BankAccount("Sandeep", "Kumawat", "987654321", 2000, False)
account3 = BankAccount("Ankit", "Sain", "123456789", 3000, True)

# Comparing instances based on account numbers
print(account1 == account2) 
print(account1 == account3)  

False
True
